In [3]:
import pandas as pd
import pandas_datareader.data as web
from datetime import datetime
import matplotlib.pyplot as plt
import yfinance as yf
import numpy as np
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.stattools import acf
from arch import arch_model

In [5]:
sp500 = yf.Ticker('^GSPC')
ge = yf.Ticker('GE')

sp500_data = yf.download('^GSPC', start='1970-01-01', end='2024-04-26')
ge_data = yf.download('GE', start='1970-01-01', end='2024-04-26')
sp500 = sp500_data['Adj Close']
ge = ge_data['Adj Close']
sp500 = sp500.loc[datetime(1970,1,1):]
ge = ge.loc[datetime(1970,1,1):]
monthly_sp500 = sp500.resample('M').last()
monthly_ge = ge.resample('M').last()
annualy_sp500 = sp500.resample('A').last()
annualy_ge = ge.resample('A').last()

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
/tmp/ipykernel_34835/295261138.py:10: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  monthly_sp500 = sp500.resample('M').last()
/tmp/ipykernel_34835/295261138.py:11: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  monthly_ge = ge.resample('M').last()
/tmp/ipykernel_34835/295261138.py:12: FutureWarning: 'A' is deprecated and will be removed in a future version, please use 'YE' instead.
  annualy_sp500 = sp500.resample('A').last()
/tmp/ipykernel_34835/295261138.py:13: FutureWarning: 'A' is deprecated and will be removed in a future version, please use 'YE' instead.
  annualy_ge = ge.resample('A').last()


In [6]:
sp500 = np.log(sp500 / sp500.shift(1))
ge = np.log(ge / ge.shift(1))
monthly_sp500 = np.log(monthly_sp500 / monthly_sp500.shift(1))
monthly_ge = np.log(monthly_ge / monthly_ge.shift(1))
annualy_sp500 = np.log(annualy_sp500 / annualy_sp500.shift(1))
annualy_ge = np.log(annualy_ge / annualy_ge.shift(1))

In [9]:
monthly_sp500.name=None
monthly_ge.name=None
monthly_sp500=pd.DataFrame(monthly_sp500,columns=['logreturn'])
monthly_sp500['realized_vol'] = realized_sp500_monthly_vol.iloc[1:]
monthly_ge = pd.DataFrame(monthly_ge,columns=['logreturn'])
monthly_ge['realized_vol'] = realized_ge_monthly_vol.iloc[1:]

NameError: name 'realized_sp500_monthly_vol' is not defined

In [7]:

def garch_1_1(data):
    garch_model = arch_model(data['logreturn'], mean='constant',vol='Garch', p=1, q=1,rescale=False)
    garch_result = garch_model.fit(update_freq=5)

    # Print out the summary of the GARCH model to see the parameters.
    print(garch_result.summary())

    # Use the model to estimate the conditional volatility.
    data['conditional_volatility'] = (garch_result.conditional_volatility)**2

    # Plot the realized volatility against the conditional volatility.
    plt.figure(figsize=(10,5))
    plt.plot(data.index, data['realized_vol'], label='Realized Volatility')
    plt.plot(data.index, data['conditional_volatility'], label='Conditional Volatility', linestyle='--')
    plt.legend()
    plt.show()

    # Calculate the RMSE for the GARCH model forecasts.
    garch_rmse = np.sqrt(np.mean(np.square(data['realized_vol'] - data['conditional_volatility'])))
    print('GARCH RMSE:', garch_rmse)
garch_1_1(monthly_sp500)

KeyError: 'logreturn'

In [8]:
monthly_sp500

Date
1970-01-31         NaN
1970-02-28    0.051352
1970-03-31    0.001451
1970-04-30   -0.094842
1970-05-31   -0.062904
                ...   
2023-12-31    0.043279
2024-01-31    0.015771
2024-02-29    0.050428
2024-03-31    0.030547
2024-04-30   -0.039981
Freq: ME, Name: Adj Close, Length: 652, dtype: float64

In [ ]:
def arch_plot(model):
    print